In [1]:
# Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from env import user,password,host

In [2]:
def get_zillow(user=user,password=password,host=host):
    """
    This function retrieves data from a MySQL database containing information about single family
    residential properties and saves it to a CSV file or returns it as a pandas dataframe.
    
    :param user: The username for the MySQL database connection
    :param password: The password is unique per person pulled from personal env
    :param host: The host parameter is the address of the server where the MySQL database is hosted
    :return: a pandas DataFrame containing information about single family residential properties in
    2017 from the Zillow database. If the data has already been saved to a CSV file, the function reads
    the data from the file. Otherwise, it reads the data from the database, saves it to a CSV file, and
    returns the DataFrame.
    """
    # name of cached csv
    filename = 'zillow2017.csv'
    # wrangle from cached data
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    # wrangle from sql db if not cached
    else:
        # read sql query into df
        # 261 is single family residential id
        df = pd.read_sql('''select bedroomcnt
                                    , bathroomcnt
                                    , calculatedfinishedsquarefeet
                                    , taxvaluedollarcnt
                                    , yearbuilt
                                    , taxamount
                                    , fips 
                                    , propertylandusetypeid
                                    , propertylandusedesc
                            from properties_2017
                            join propertylandusetype using(propertylandusetypeid)
                            where propertylandusetypeid = 261'''
                            , f'mysql+pymysql://{user}:{password}@{host}/zillow')
        # cache data locally
        df.to_csv(filename, index=False)
        return df

In [3]:
# get initial data
df = get_zillow()

In [4]:
# check size
df.shape

(2152863, 9)

In [5]:
# look at value ranges
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
yearbuilt,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00
propertylandusetypeid,2152863.0,261.000000,0.000000,261.00,261.00,261.00,261.00,261.00


In [6]:
# look at dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 9 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
 7   propertylandusetypeid         float64
 8   propertylandusedesc           object 
dtypes: float64(8), object(1)
memory usage: 147.8+ MB


In [7]:
# check and count nulls
df.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
propertylandusetypeid              0
propertylandusedesc                0
dtype: int64

In [8]:
# what if we drop, what is the size
df1 = df.dropna()
df1.shape

(2140235, 9)

In [9]:
# how much does it account for
df1.shape[0] / df.shape[0]

0.9941343225277224

In [10]:
# less than 1% so dropping
df = df.dropna()

In [11]:
import wrangle

In [12]:
df2 = wrangle.wrangle_zillow()
df2

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid,propertylandusedesc
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,261.0,Single Family Residential
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0,261.0,Single Family Residential
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0,261.0,Single Family Residential
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0,261.0,Single Family Residential
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0,261.0,Single Family Residential
...,...,...,...,...,...,...,...,...,...
2152856,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0,261.0,Single Family Residential
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0,261.0,Single Family Residential
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0,261.0,Single Family Residential
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0,261.0,Single Family Residential
